In [1]:
import numpy as np
import pandas as pd

In [2]:
# We read in a stock data data file into a data frame and see what it looks like
zurn_data = pd.read_csv("ZURN.SW.csv")

# We display the first 5 rows of the DataFrame
print(zurn_data.head(5))

         Date        Open        High         Low       Close   Adj Close  \
0  2018-08-13  296.899994  298.700012  294.899994  296.700012  279.776276   
1  2018-08-14  299.100006  300.000000  294.899994  296.500000  279.587677   
2  2018-08-15  296.000000  297.399994  293.799988  294.799988  277.984619   
3  2018-08-16  296.000000  297.799988  295.700012  296.700012  279.776276   
4  2018-08-17  298.299988  298.299988  293.799988  295.500000  278.644714   

   Volume  
0  572052  
1  350106  
2  306225  
3  414659  
4  331706  


In [3]:
stock_data = pd.read_csv("ZURN.SW.csv", index_col="Date", parse_dates=True, usecols=['Date','Adj Close'])
print(stock_data.head(5))

             Adj Close
Date                  
2018-08-13  279.776276
2018-08-14  279.587677
2018-08-15  277.984619
2018-08-16  279.776276
2018-08-17  278.644714


In [4]:
stock_data.describe()

,Adj Close
count,254.000000
mean,307.319245
std,22.913125
min,267.894958
25%,287.791443
50%,301.558640
75%,325.600006
max,351.799988


In [5]:
print("Vol in $:",stock_data.std()[0])
vol = 100*(stock_data.std()[0]/stock_data.mean()[0])
print("Vol used for pricing in % terms ",vol)

('Vol in $:', 22.91312515331088)
('Vol used for pricing in % terms ', 7.4558054861334435)


In [6]:
print('We assume that the normal trading days in a year is 252')
t = float(raw_input('Enter days to maturity:'))
t = float(t/252)
print(t)

We assume that the normal trading days in a year is 252
Enter days to maturity:92
0.365079365079


In [7]:
r = float(raw_input('Enter risk free rate as a decimal:'))

Enter risk free rate as a decimal:0


In [8]:
q = float(raw_input('How much dividend does the stock pay? '))

How much dividend does the stock pay? 0


In [9]:
S_0 = stock_data['Adj Close'][-1]
print S_0

351.799988


In [10]:
X = float(raw_input('Enter the strike price: '))

Enter the strike price: 340


In [14]:
#Calculate U (factor of up-move in price at expiration)
U = np.power(np.e, (vol/100)*np.power(t, .5))
print("U=",U)
#Calculate D (factor of down-move in price at expiration)
D = 1/U
print("D=",D)

('U=', 1.0460794483047888)
('D=', 0.9559503359142919)


In [15]:
#Calculate the probability of U
P_U= (np.power(np.e, (r)*t) - D)/(U-D)
print("P_U=",P_U)
#Calculate the probability of D
P_D = 1 - P_U
print("P_D=",P_D)

('P_U=', 0.4887395750094242)
('P_D=', 0.5112604249905758)


In [16]:
#Calculate the up-move/increase in stock price at expiry
S_U = S_0*U
print("S_U=",S_U)
#Calculate the down-move/decrease in stock price at expiry
S_D = S_0*D
print("S_D=",S_D)

('S_U=', 368.0107373606713)
('S_D=', 336.30331670324387)


In [17]:
#Calculate call option payoff
C = max(0, P_D*(S_D-X), P_U*(S_U-X))
print("C=",C)

#Calculate call option value today
c = C*np.power(np.e, -r*t)
print("c=",c)

('C=', 13.689955873355107)
('c=', 13.689955873355107)


In [20]:
stock = np.zeros([100,100])
print stock

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [22]:
def binomial_call_pricer(N, S_0, v, r, X, t, q):
    """
    
    This pricer takes as input the variables below which we have already defined above based on an N step binomial pricing model
    and returns the price of a call option.
    
    Inputs:
    N (int) = number of binomial steps
    S_0 (float) = initial stock price
    v (float) = annual volatility
    r (float) = risk free interest rate per annum
    X (float) = strike price
    t (float) = time to expiration
    q (float) = dividend yield
    
    Returns (float): call price in local currency terms
    
    """
    #Calculate the up-move and down-move factors and associated probabilities
    U = np.power(np.e, (vol/100)*np.power(t, .5))
    D = 1 / U
    P_U = (np.power(np.e, (r)*t) - D)/(U-D)
    P_D = 1 - P_U
 
    # make stock price tree
    stock = np.zeros([N + 1, N + 1])
    for i in range(N + 1):
        for j in range(i + 1):
            stock[j, i] = S_0 * np.power(U,i - j) * np.power(D,j)
            
    # Generate option prices recursively                          
    option= np.zeros([N + 1, N + 1])
    option[:, N] = np.maximum(np.zeros(N + 1), (stock[:, N] - X))
    for i in range(N - 1, -1, -1):
        for j in range(0, i + 1):
            option[j, i] = (np.power(np.e, -r*t)* (P_U * option[j, i + 1] + P_D * option[j + 1, i + 1]))
           
            
    return option[0,0]

N = int(raw_input('How many steps do you want to use? '))
                                 
print("Call price for {} steps is {}".format(N,binomial_call_pricer(N, S_0, vol/100, r, X, t, q)))

How many steps do you want to use? 120
Call price for 120 steps is 73.5897664177
